### Mount Google Drive

Mount your Google Drive on your runtime using an authorization code.

Follow the instruction on the screen to authorise Colab accessing your drive. On your Google Drive, you can access you files prepend with “/content/drive/MyDrive/”

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change working director
%cd "/content/drive/MyDrive/Colab Notebooks/goML"

/content/drive/MyDrive/Colab Notebooks/goML


### Import necessary libraries and dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
names = ['preg' , 'plas' , 'pres' , 'skin' , 'test' , 'mass' , 'pedi' , 'age' , 'class' ]
#Load our data set
df = pd.read_csv("data/diabetes.csv",names=names)

In [ ]:
df

preg  plas  pres  skin  test  mass   pedi  age  class
0       6   148    72    35     0  33.6  0.627   50      1
1       1    85    66    29     0  26.6  0.351   31      0
2       8   183    64     0     0  23.3  0.672   32      1
3       1    89    66    23    94  28.1  0.167   21      0
4       0   137    40    35   168  43.1  2.288   33      1
..    ...   ...   ...   ...   ...   ...    ...  ...    ...
763    10   101    76    48   180  32.9  0.171   63      0
764     2   122    70    27     0  36.8  0.340   27      0
765     5   121    72    23   112  26.2  0.245   30      0
766     1   126    60     0     0  30.1  0.349   47      1
767     1    93    70    31     0  30.4  0.315   23      0

[768 rows x 9 columns]

In [ ]:
array = df.values
X = array[:,0:8]
Y = array[:,8]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

In [ ]:
model = LogisticRegression(max_iter=400)
model.fit(X_train, Y_train)


LogisticRegression(max_iter=400)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(Y_test, y_pred)
cnf_matrix

array([[142,  20],
       [ 34,  58]])

In [ ]:
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))
print("Precision:",metrics.precision_score(Y_test, y_pred))
print("Recall:",metrics.recall_score(Y_test, y_pred))

Accuracy: 0.7874015748031497
Precision: 0.7435897435897436
Recall: 0.6304347826086957


Save model and reload for future use

In [ ]:
import joblib
# Save the trained model to a file so we can use it to make predictions later
joblib.dump(model, 'models/diabetes_model.pkl')

['models/diabetes_model.pkl']

In [ ]:
# Load our trained model
model = joblib.load('models/diabetes_model.pkl')

In [ ]:
y_pred = model.predict(X_test)
cnf_matrix = metrics.confusion_matrix(Y_test, y_pred)
cnf_matrix

array([[142,  20],
       [ 34,  58]])

### Make inference

In [ ]:
# Define the house that we want to value (with the values in the same order as in the training data)
patient_1 = [
    6,  # preg
    148,  # plas
    72,  # pres
    35,  # skin
    0,  # test
    33.6,  # mass
    0.627,  # pedi
    50,  # age
]


In [ ]:
# scikit-learn assumes you want to predict the values for multiple of houses at once, so it expects an array.
# We only want to estimate the value of a single house, so there will only be one item in our array.
patients = [
    patient_1
]

In [ ]:
# Make a prediction for each house in the homes array (we only have one)
prep_values = model.predict(patients)

# Since we are only predicting the price of one house, grab the first prediction returned
predicted_value = prep_values[0]

In [ ]:
# Print the results
print("Predicated class: %s"%predicted_value)

Predicated class: 1.0


### Apply standardisation

In [ ]:
from sklearn.preprocessing import StandardScaler
# Step 3: apply pre-processing transform to the input variables
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(rescaledX, Y, test_size=0.33, random_state=7)

In [ ]:
model = LogisticRegression(max_iter=400)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=400)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
cnf_matrix = metrics.confusion_matrix(Y_test, y_pred)
cnf_matrix

array([[141,  21],
       [ 34,  58]])

In [ ]:
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))
print("Precision:",metrics.precision_score(Y_test, y_pred))
print("Recall:",metrics.recall_score(Y_test, y_pred))

Accuracy: 0.7834645669291339
Precision: 0.7341772151898734
Recall: 0.6304347826086957


### Using k-fold

In [ ]:
  
num_folds = 10
kfold = KFold(n_splits=5, shuffle=True)
model = LogisticRegression(max_iter=400)
results = cross_val_score(model, X, Y, cv=kfold)
print("Accuracy: %.3f%%" % (results.mean()*100.0))

Accuracy: 77.216%


Note: Convergent warning
https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter



### Exercise

Make a prediction when a patient has the following data:

    7,  # preg
    132,  # plas
    80,  # pres
    30,  # skin
    0,  # test
    45.5,  # mass
    0.547,  # pedi
    45,  # age
